In [ ]:

import keras
import tensorflow as tf
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Lambda, MaxPool2D, BatchNormalization
from keras.utils import np_utils
from keras.utils import model_to_dot
from tensorflow.keras.utils import plot_model
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import models, layers, optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.utils import class_weight
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta, RMSprop
from keras.models import Sequential, model_from_json
from keras.layers import Activation,Dense, Dropout, Flatten, Conv2D, MaxPool2D,MaxPooling2D,AveragePooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.layers.merge import concatenate
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
import os
from glob import glob
import matplotlib.pyplot as plt
import random
import cv2
import pandas as pd
import numpy as np
import matplotlib.gridspec as gridspec
import seaborn as sns
import zlib
import itertools
import sklearn
import itertools
import scipy
import skimage
from skimage.transform import resize
import csv
from tqdm import tqdm
from sklearn import model_selection
from sklearn.model_selection import train_test_split, learning_curve,KFold,cross_val_score,StratifiedKFold
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix

#ROC Curve
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from scipy import interp
from itertools import cycle
from sklearn.metrics import roc_curve, auc

In [ ]:
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

class MetricsCheckpoint(Callback):
    def __init__(self, savepath):
        super(MetricsCheckpoint, self).__init__()
        self.savepath = savepath
        self.history = {}
    def on_epoch_end(self, epoch, logs=None):
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        np.save(self.savepath, self.history)

def plotKerasLearningCurve():
    plt.figure(figsize=(10,5))
    metrics = np.load('logs.npy')[()]
    filt = ['acc'] # try to add 'loss' to see the loss learning curve
    for k in filter(lambda x : np.any([kk in x for kk in filt]), metrics.keys()):
        l = np.array(metrics[k])
        plt.plot(l, c= 'r' if 'val' not in k else 'b', label='val' if 'val' in k else 'train')
        x = np.argmin(l) if 'loss' in k else np.argmax(l)
        y = l[x]
        plt.scatter(x,y, lw=0, alpha=0.25, s=100, c='r' if 'val' not in k else 'b')
        plt.text(x, y, '{} = {:.4f}'.format(x,y), size='15', color= 'r' if 'val' not in k else 'b')   
    plt.legend(loc=4)
    plt.axis([0, None, None, None]);
    plt.grid()
    plt.xlabel('Number of epochs')
    plt.ylabel('Accuracy')

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize = (8,8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def plot_learning_curve(history):
    plt.figure(figsize=(8,8))
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./accuracy_curve.png')
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./loss_curve.png')

#ROC Curve
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from scipy import interp
from itertools import cycle
from sklearn.metrics import roc_curve, auc

In [ ]:

# Defining Hyperparameters and model structure
image_size=224
target_dims = (image_size,image_size,3) # add channel for RGB
number_classes = 2
number_epoch=150
batch_size=16
learningRate=0.0001
patience=100
# optimizer = keras.optimizers.Adam(lr=0.001)
optimizer = keras.optimizers.RMSprop(lr=learningRate)



In [ ]:
# create data map  (you can skip data map generation if you have already had array data . Then, they we'll be loaded in the following))

dir_Frontal = "/content/drive/My Drive/HRV-workplace/HRV_d/Ankle_fx/Categorized/Frontal/"
dir_Lateral = "/content/drive/My Drive/HRV-workplace/HRV_d/Ankle_fx/Categorized/Lateral/"
dir_Oblique = "/content/drive/My Drive/HRV-workplace/HRV_d/Ankle_fx/Categorized/Oblique/"



from tqdm import tqdm
def get_data(folder):
    """
    Load the data and labels from the given folder.
    """
    X = []
    y = []
    for folderName in os.listdir(folder):
        if not folderName.startswith('.'):
            if folderName in ['Patient']:
                label = 0
            elif folderName in ['Normal']:
                label = 1



            for image_filename in tqdm(os.listdir(folder + folderName)):

                img_file = cv2.imread(folder + folderName + '/' + image_filename)
                if img_file is not None:
                    img_file = skimage.transform.resize(img_file, (image_size, image_size))
                    #repeate grayscale image to create RGB- style shpae(*,*,3)
                    img_file=img_file/224.0####################################################################change it by imagesize.0
                    # img_file=np.dstack([img_file] * 3)
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y
X_Frontal, Y_Frontal = get_data(dir_Frontal) 
X_Lateral, Y_Lateral = get_data(dir_Lateral) 
X_Oblique, Y_Oblique = get_data(dir_Oblique) 




In [ ]:
# # # save data in array foramt


# np.save('/content/drive/My Drive/HRV-workplace/HRV_d/Ankle_fx/Vector/X_Frontal', X_Frontal)
# np.save('/content/drive/My Drive/HRV-workplace/HRV_d/Ankle_fx/Vector/X_Lateral', X_Lateral)
# np.save('/content/drive/My Drive/HRV-workplace/HRV_d/Ankle_fx/Vector/X_Oblique', X_Oblique)


# np.save('/content/drive/My Drive/HRV-workplace/HRV_d/Ankle_fx/Vector/Y_Frontal', Y_Frontal)
# # or
# np.savetxt('/content/drive/My Drive/HRV-workplace/HRV_d/Ankle_fx/Vector/Y_Frontal.txt', Y_Frontal, fmt='%d')








 
# # load data in array format (you can skip data map generation if you have already had array data)

X_Frontal=np.load('/content/drive/My Drive/HRV-workplace/HRV_d/Ankle_fx/Vector/X_Frontal.npy')
X_Lateral=np.load('/content/drive/My Drive/HRV-workplace/HRV_d/Ankle_fx/Vector/X_Lateral.npy')
X_Oblique=np.load('/content/drive/My Drive/HRV-workplace/HRV_d/Ankle_fx/Vector/X_Oblique.npy')


Y_Frontal=np.load('/content/drive/My Drive/HRV-workplace/HRV_d/Ankle_fx/Vector/Y_Frontal.npy')
# or
# Y_Frontal=np.loadtxt('/content/drive/My Drive/HRV-workplace/HRV_d/Series1/Vector/Y_Frontal.txt',dtype=int)

# # Y_Frontal=Y



In [ ]:


labels = {0: 'Patient',  1: 'Normal'}
# dict_characters=labels
# import seaborn as sns
# df = pd.DataFrame()
# df["labels"]=Y_Frontal
# lab = df['labels']
# dist = lab.value_counts()
# sns.countplot(lab)
# print(dict_characters)

In [ ]:
#Generate Model 

from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50

from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

def Model_build():
  # Importing the important libraries

  def single_branch(name_modifier):#add name_modifier cause we use a single model(like InceptionV3) mutilple times. 
    
    base_model = InceptionV3(include_top=False, weights='imagenet')
    for layer in base_model.layers:
      layer.trainable = True
    
    for layer in base_model.layers:
      layer._name = layer.name + str(name_modifier)


    x = base_model.output
    
    x = GlobalAveragePooling2D()(x)

    # x = Dense(1024, activation='relu')(x)

    # predictions = Dense(number_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=x)
    
    # Training only top layers i.e. the layers which we have added in the end
    return model






  Frontal_branch=single_branch('one')
  Lateral_branch=single_branch('two')
  Oblique_branch=single_branch('three')



  # plot_model(Oblique_branch)


  combinedInput = concatenate([Frontal_branch.output, Lateral_branch.output,Oblique_branch.output])

  predictions = Dense(number_classes, activation='softmax',name='visualize_layer')(combinedInput)



  model=Model(inputs=[Frontal_branch.input,Lateral_branch.input,Oblique_branch.input],outputs=predictions)
  return model

# model.summary()

# plot_model(model)






  # model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])
  




In [ ]:
model_score=[]

for i in range(10):


  for train_index,val_index in KFold(5, shuffle=True).split(X_Frontal):

    X_Frontal_train,X_Frontal_test=X_Frontal[train_index],X_Frontal[val_index]
    X_Lateral_train,X_Lateral_test=X_Lateral[train_index],X_Lateral[val_index]
    X_Oblique_train,X_Oblique_test=X_Oblique[train_index],X_Oblique[val_index]

    y_train,y_test=Y_Frontal[train_index],Y_Frontal[val_index]

    break




  for train_index,val_index in KFold(4, shuffle=True).split(X_Frontal_train):

    X_Frontal_train,X_Frontal_val=X_Frontal_train[train_index],X_Frontal_train[val_index]
    X_Lateral_train,X_Lateral_val=X_Lateral_train[train_index],X_Lateral_train[val_index]
    X_Oblique_train,X_Oblique_val=X_Oblique_train[train_index],X_Oblique_train[val_index]

    y_train,y_val=y_train[train_index],y_train[val_index]


    break



  # one-hot encoding
  from keras.utils.np_utils import to_categorical
  y_trainHot = to_categorical(y_train, num_classes = number_classes)
  y_valhot = to_categorical(y_val, num_classes = number_classes)
  y_testhot = to_categorical(y_test, num_classes = number_classes)





  #     layer.trainable = True


  model=Model_build()
  model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])



  # We will try to train the last stage of InceptionV3
  filepath='/content/drive/My Drive/HRV-workplace/5-AnkleFx/Model/Ankle_fx_3sides{}.h5'.format(i)
  checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max',save_weights_only=True)
  callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=patience, verbose=2),checkpoint]
  # callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=patience, verbose=2)]


  # Training the model for 10 epochs
  # history = model.fit(x_train,y_trainHot, epochs=number_epoch, validation_data=(x_test,y_testhot) ,verbose=1 ,callbacks = callbacks_list)



  history=model.fit([X_Frontal_train, X_Lateral_train,X_Oblique_train], y_trainHot,
  validation_data=([X_Frontal_val, X_Lateral_val,X_Oblique_val], y_valhot),
  epochs=number_epoch, batch_size=batch_size,callbacks = callbacks_list,verbose=1)


  model=Model_build()
  model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])

  model.load_weights(filepath)



  print('****** Train set evaluation .....\n ')
  # Evaluate model on train data
  score = model.evaluate([X_Frontal_train, X_Lateral_train,X_Oblique_train],y_trainHot, verbose=0)
  print('\nKeras CNN - accuracy:', score[1], '\n')
  y_pred = model.predict([X_Frontal_train, X_Lateral_train,X_Oblique_train])
  confusion_mtx=confusion_matrix(y_train, np.argmax(y_pred, axis=1))
  print('\n', sklearn.metrics.classification_report(np.where(y_trainHot > 0)[1], np.argmax(y_pred, axis=1), target_names=list(labels.values())), sep='') 


  # plt.show()
  # plot_confusion_matrix(confusion_mtx, classes = list(labels.values()))
  # plt.show()




  print('****** Val set evaluation .....\n ')
  # Evaluate model on test data
  scores = model.evaluate([X_Frontal_val, X_Lateral_val,X_Oblique_val],y_valhot, verbose=0)
  print('\nKeras CNN - accuracy:', scores[1], '\n')
  y_pred = model.predict([X_Frontal_val, X_Lateral_val,X_Oblique_val])
  confusion_mtx=confusion_matrix(y_val, np.argmax(y_pred, axis=1))
  print('\n', sklearn.metrics.classification_report(np.where(y_valhot > 0)[1], np.argmax(y_pred, axis=1), target_names=list(labels.values())), sep='') 






  print('****** Test set evaluation .....\n ')
  # Evaluate model on test data
  x_test=[X_Frontal_test, X_Lateral_test,X_Oblique_test]
  scores = model.evaluate([X_Frontal_test, X_Lateral_test,X_Oblique_test],y_testhot, verbose=0)
  print('\nKeras CNN - accuracy:', scores[1], '\n')
  y_pred = model.predict([X_Frontal_test, X_Lateral_test,X_Oblique_test])
  confusion_mtx=confusion_matrix(y_test, np.argmax(y_pred, axis=1))
  print('\n', sklearn.metrics.classification_report(np.where(y_testhot > 0)[1], np.argmax(y_pred, axis=1), target_names=list(labels.values())), sep='') 








  print('****** Detail Test set evaluation .....\n ')
  from sklearn.metrics import precision_recall_fscore_support,roc_curve,auc
  detail_result=precision_recall_fscore_support(np.where(y_testhot > 0)[1], np.argmax(y_pred, axis=1))
  Specificity=confusion_mtx[1,1]/(confusion_mtx[1,1]+confusion_mtx[1,0])


  fpr1, tpr1, thresholds =roc_curve(y_test, y_pred[:,1])
  auc_result=auc(fpr1, tpr1)

  print("Precision={}\nRecall={}\nSpecificity{}\nFscore={}".format(detail_result[0][0],detail_result[1][0],Specificity,detail_result[2][0]))
  print('auc={}'.format(auc_result))


  plot_learning_curve(history)

  plt.show()
  plot_confusion_matrix(confusion_mtx, classes = list(labels.values()))
  plt.show()

  


  model_score.append(scores[1] * 100)




















  #ROC

    # Binarize the output
  y_bin = label_binarize(y_testhot, classes=[0, 1])
  n_classes = number_classes
  lw=number_classes

  y_score=model.predict(x_test)

  fpr = dict()
  tpr = dict()
  roc_auc = dict()
  for i in range(n_classes):
      fpr[i], tpr[i], _ = roc_curve(y_bin[:, i], y_score[:, i])
      roc_auc[i] = auc(fpr[i], tpr[i])
  colors = cycle(['blue', 'red', 'green'])

  print(roc_auc)
  # for i, color in zip(range(n_classes), colors):
  plt.plot(fpr[0], tpr[0], color='red', lw=lw,
              label='Pre-trained Inceptiov-v3 (area = {:0.2f})'
              ''.format(roc_auc[0]))





  plt.plot([0, 1], [0, 1], 'k--', lw=lw)
  plt.xlim([-0.01, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  # plt.title('Receiver operating characteristic for multi-class data')
  plt.legend(loc="lower right")
  plt.show()


























print("%.2f%% (+/- %.2f%%)" % (np.mean(model_score), np.std(model_score)))









**ResNet50**

In [ ]:
#Generare Model Resnet

from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50

from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

def Model_build_resnet():

  base_model = ResNet50(include_top=False, weights='imagenet')
  # Training only top layers i.e. the layers which we have added in the end
  for layer in base_model.layers:
      layer.trainable = True
  
  # Taking the output of the last convolution block in InceptionV3
  x = base_model.output
  
  # Adding a Global Average Pooling layer
  x = GlobalAveragePooling2D()(x)



  # Adding a fully connected layer having 2 neurons which will
  # give the probability of image having either dog or cat
  predictions = Dense(2, activation='softmax')(x)
  
  # Model to be trained
  model = Model(inputs=base_model.input, outputs=predictions)
    
    # Training only top layers i.e. the layers which we have added in the end
  return model



In [ ]:

# Defining Hyperparameters and model structure
image_size=224
target_dims = (image_size,image_size,3) # add channel for RGB
number_classes = 2
number_epoch=200
batch_size=32
learningRate=0.0001
patience=200
# optimizer = keras.optimizers.Adam(lr=0.001)
optimizer = keras.optimizers.RMSprop(lr=learningRate)


model_score=[]

for i in range(10):


  for train_index,val_index in KFold(5, shuffle=True).split(X_Frontal):

    X_Frontal_train,X_Frontal_test=X_Frontal[train_index],X_Frontal[val_index]
    X_Lateral_train,X_Lateral_test=X_Lateral[train_index],X_Lateral[val_index]
    X_Oblique_train,X_Oblique_test=X_Oblique[train_index],X_Oblique[val_index]

    y_train,y_test=Y_Frontal[train_index],Y_Frontal[val_index]

    break




  for train_index,val_index in KFold(4, shuffle=True).split(X_Frontal_train):

    X_Frontal_train,X_Frontal_val=X_Frontal_train[train_index],X_Frontal_train[val_index]
    X_Lateral_train,X_Lateral_val=X_Lateral_train[train_index],X_Lateral_train[val_index]
    X_Oblique_train,X_Oblique_val=X_Oblique_train[train_index],X_Oblique_train[val_index]

    y_train,y_val=y_train[train_index],y_train[val_index]


    break



  # one-hot encoding
  from keras.utils.np_utils import to_categorical
  y_trainHot = to_categorical(y_train, num_classes = number_classes)
  y_valhot = to_categorical(y_val, num_classes = number_classes)
  y_testhot = to_categorical(y_test, num_classes = number_classes)





  #     layer.trainable = True


  model=Model_build_resnet()
  model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])



  # We will try to train the last stage of InceptionV3

  filepath='/content/drive/My Drive/HRV-workplace/5-AnkleFx/Model/Ankle_fx_3sides_resnet.h5'

  checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max',save_weights_only=True)
  callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=patience, verbose=2),checkpoint]
  # callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=patience, verbose=2)]


  # Training the model for 10 epochs
  # history = model.fit(x_train,y_trainHot, epochs=number_epoch, validation_data=(x_test,y_testhot) ,verbose=1 ,callbacks = callbacks_list)



  history=model.fit([X_Frontal_train, X_Lateral_train,X_Oblique_train], y_trainHot,
  validation_data=([X_Frontal_val, X_Lateral_val,X_Oblique_val], y_valhot),
  epochs=number_epoch, batch_size=batch_size,callbacks = callbacks_list,verbose=1)


  model=Model_build_resnet()
  model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])

  model.load_weights(filepath)



  print('****** Train set evaluation .....\n ')
  # Evaluate model on train data
  score = model.evaluate([X_Frontal_train, X_Lateral_train,X_Oblique_train],y_trainHot, verbose=0)
  print('\nKeras CNN - accuracy:', score[1], '\n')
  y_pred = model.predict([X_Frontal_train, X_Lateral_train,X_Oblique_train])
  confusion_mtx=confusion_matrix(y_train, np.argmax(y_pred, axis=1))
  print('\n', sklearn.metrics.classification_report(np.where(y_trainHot > 0)[1], np.argmax(y_pred, axis=1), target_names=list(labels.values())), sep='') 


  # plt.show()
  # plot_confusion_matrix(confusion_mtx, classes = list(labels.values()))
  # plt.show()




  print('****** Val set evaluation .....\n ')
  # Evaluate model on test data
  scores = model.evaluate([X_Frontal_val, X_Lateral_val,X_Oblique_val],y_valhot, verbose=0)
  print('\nKeras CNN - accuracy:', scores[1], '\n')
  y_pred = model.predict([X_Frontal_val, X_Lateral_val,X_Oblique_val])
  confusion_mtx=confusion_matrix(y_val, np.argmax(y_pred, axis=1))
  print('\n', sklearn.metrics.classification_report(np.where(y_valhot > 0)[1], np.argmax(y_pred, axis=1), target_names=list(labels.values())), sep='') 






  print('****** Test set evaluation .....\n ')
  # Evaluate model on test data
  x_test=[X_Frontal_test, X_Lateral_test,X_Oblique_test]
  scores = model.evaluate([X_Frontal_test, X_Lateral_test,X_Oblique_test],y_testhot, verbose=0)
  print('\nKeras CNN - accuracy:', scores[1], '\n')
  y_pred = model.predict([X_Frontal_test, X_Lateral_test,X_Oblique_test])
  confusion_mtx=confusion_matrix(y_test, np.argmax(y_pred, axis=1))
  print('\n', sklearn.metrics.classification_report(np.where(y_testhot > 0)[1], np.argmax(y_pred, axis=1), target_names=list(labels.values())), sep='') 








  print('****** Detail Test set evaluation .....\n ')
  from sklearn.metrics import precision_recall_fscore_support,roc_curve,auc
  detail_result=precision_recall_fscore_support(np.where(y_testhot > 0)[1], np.argmax(y_pred, axis=1))
  Specificity=confusion_mtx[1,1]/(confusion_mtx[1,1]+confusion_mtx[1,0])


  fpr1, tpr1, thresholds =roc_curve(y_test, y_pred[:,1])
  auc_result=auc(fpr1, tpr1)

  print("Precision={}\nRecall={}\nSpecificity{}\nFscore={}".format(detail_result[0][0],detail_result[1][0],Specificity,detail_result[2][0]))
  print('auc={}'.format(auc_result))


  plot_learning_curve(history)

  plt.show()
  plot_confusion_matrix(confusion_mtx, classes = list(labels.values()))
  plt.show()

  


  model_score.append(scores[1] * 100)



















  #ROC

    # Binarize the output
  y_bin = label_binarize(y_testhot, classes=[0, 1])
  n_classes = number_classes
  lw=number_classes

  y_score=model.predict(x_test)

  fpr = dict()
  tpr = dict()
  roc_auc = dict()
  for i in range(n_classes):
      fpr[i], tpr[i], _ = roc_curve(y_bin[:, i], y_score[:, i])
      roc_auc[i] = auc(fpr[i], tpr[i])
  colors = cycle(['blue', 'red', 'green'])

  print(roc_auc)
  # for i, color in zip(range(n_classes), colors):
  plt.plot(fpr[0], tpr[0], color='red', lw=lw,
              label='Pre-trained Inceptiov-v3 (area = {:0.2f})'
              ''.format(roc_auc[0]))





  plt.plot([0, 1], [0, 1], 'k--', lw=lw)
  plt.xlim([-0.01, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  # plt.title('Receiver operating characteristic for multi-class data')
  plt.legend(loc="lower right")
  plt.show()


























print("%.2f%% (+/- %.2f%%)" % (np.mean(model_score), np.std(model_score)))







